# Korean LLM (Large Language Model) Inference
---

### Model: [KKULM-Polyglot-12.8B](https://huggingface.co/nlpai-lab/kullm-polyglot-12.8b-v2)

- LLM GitHub: https://github.com/nlpai-lab/KULLM
- Hugging Face model hub: https://huggingface.co/nlpai-lab/kullm-polyglot-12.8b-v2

In [5]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../utils')
sys.path.append('../templates')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
!pip install -qU boto3 huggingface_hub sagemaker langchain deepspeed 
!pip install -qU bitsandbytes accelerate peft

In [7]:
import os
from pathlib import Path
from huggingface_hub import snapshot_download

HF_MODEL_ID = "nlpai-lab/kullm-polyglot-12.8b-v2"

# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.bin", "*.txt", "*.model"]

# create model dir
model_name = HF_MODEL_ID.split("/")[-1].replace('.', '-')
model_tar_dir = Path(f"/home/ec2-user/SageMaker/models/{model_name}")
if not os.path.isdir(model_tar_dir):
    os.makedirs(model_tar_dir, exist_ok=True)
    # Download model from Hugging Face into model_dir
    snapshot_download(
        HF_MODEL_ID, 
        local_dir=str(model_tar_dir), 
        local_dir_use_symlinks=False,
        allow_patterns=allow_patterns,
        cache_dir="/home/ec2-user/SageMaker/"
    )

<img src="./images/Slide3.png"/>

<img src="./images/Slide4.png"/>

<img src="./images/Slide5.png"/>

In [8]:
import torch
from transformers import BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [9]:
import torch
import deepspeed
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, GPTNeoXLayer

model = AutoModelForCausalLM.from_pretrained(
    model_tar_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    #cache_dir=local_model_path,
    quantization_config=bnb_config
)

[2023-09-06 02:14:02,227] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
model_fp16 = AutoModelForCausalLM.from_pretrained(
    model_tar_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
model_fp16.get_memory_footprint()

25954985040

In [14]:
model.get_memory_footprint()

7080614480

<img src="./images/Slide6.png"/>

In [15]:
# config = {
#     "tensor_parallel": {"tp_size": 1},
#     "dtype": "fp16",
#     "injection_policy": {
#         GPTNeoXLayer:('attention.dense', 'mlp.dense_4h_to_h')
#     }
# }

# modl = deepspeed.init_inference(model, config)

local_rank = int(os.getenv('LOCAL_RANK', '0'))
tokenizer = AutoTokenizer.from_pretrained(HF_MODEL_ID)
generator = pipeline(
    task="text-generation", model=model, tokenizer=tokenizer
)

In [16]:
import json
from inference_lib import Prompter
prompter = Prompter("kullm")

params = {
    "do_sample": False,
    "max_new_tokens": 256,
    "return_full_text": True,
    "temperature": 0.01,
    "top_p": 0.9,
    "return_full_text": False,
    "repetition_penalty": 1.1,
    "presence_penalty": None,
    "eos_token_id": 2,
}

instruction = "아래 질문에 100글자 이상으로 자세하게 대답해줘."
#instruction = ""
input_text = "고려대학교에 대해서 알려줘"
prompt = prompter.generate_prompt(instruction, input_text)
payload = {
    "inputs": [prompt,],
    "parameters": params
}

In [18]:
%%timeit -n3 -r1
text_inputs, params = payload["inputs"], payload["parameters"]
result = generator(text_inputs, **params)
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[[{'generated_text': '고려대학교는 한국의 명문 사립대학으로, 1931년 5월 1일에 설립되었습니다. 고려대학교는 한국에서 가장 오래된 고등 교육 기관 중 하나로, 한국의 역사와 문화적 전통을 이어가고 있습니다. 고려대학교는 학문과 연구 분야에서 뛰어난 성과를 거두며 한국 사회 발전에 기여하고 있으며, 다양한 분야에서 활약하는 동문들을 배출했습니다. 고려대학교는 학생들이 자신의 잠재력을 최대한 발휘할 수 있도록 지원하기 위해 최선을 다하고 있으며, 학생들에게 다양한 기회와 자원을 제공합니다.'}]]


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[[{'generated_text': '고려대학교는 한국의 명문 사립대학으로, 1931년 5월 1일에 설립되었습니다. 고려대학교는 한국에서 가장 오래된 고등 교육 기관 중 하나로, 한국의 역사와 문화적 전통을 이어가고 있습니다. 고려대학교는 학문과 연구 분야에서 뛰어난 성과를 거두며 한국 사회 발전에 기여하고 있으며, 다양한 분야에서 활약하는 동문들을 배출했습니다. 고려대학교는 학생들이 자신의 잠재력을 최대한 발휘할 수 있도록 지원하기 위해 최선을 다하고 있으며, 학생들에게 다양한 기회와 자원을 제공합니다.'}]]
[[{'generated_text': '고려대학교는 한국의 명문 사립대학으로, 1931년 5월 1일에 설립되었습니다. 고려대학교는 한국에서 가장 오래된 고등 교육 기관 중 하나로, 한국의 역사와 문화적 전통을 이어가고 있습니다. 고려대학교는 학문과 연구 분야에서 뛰어난 성과를 거두며 한국 사회 발전에 기여하고 있으며, 다양한 분야에서 활약하는 동문들을 배출했습니다. 고려대학교는 학생들이 자신의 잠재력을 최대한 발휘할 수 있도록 지원하기 위해 최선을 다하고 있으며, 학생들에게 다양한 기회와 자원을 제공합니다.'}]]
8.96 s ± 0 ns per loop (mean ± std. dev. of 1 run, 3 loops each)
